## Notebook 1 Some Feature Engineering and Size Reduction

In this notebook we will take the original dataset and perform the necessary manipulations that will allow us to work with it in a more efficient manner. Therefore, we will simply focus on:

#### 1. Removing all unnecessary text

#### 2. As the data is organized into rows with values and rows number of items sold (and there are 2 rows per entry, we can actually store the value as a column, thus reducing the dataset's size to about half)

#### 3. Additionally, we created 3 additional features:
 
a. Unit price (the quotient between value by which the product was sold) and quantity sold.

b. Retail price (assumed to be current price by which is product is supposed to be sold if there are no special offers. While unlikely, it is possible that different stores practice different prices altogether (therefore this was computed as the max unit price of each SKU in each Store).

c. Is Promo - It assumes that, if a product was sold at a unit price below, at least, 10% of the considered retail price, then the product was sold at a promotional price. This choice cannot capture promotions under 10% but has other advantages: if a products price increased due to inflation, it does not consider that all previous sales were promos.

In [1]:
#import libs

import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')

In [2]:
goliath = pd.read_csv('../Databases/The Goliath.csv')
goliath

,ProductFamily_ID,ProductCategory_ID,ProductBrand_ID,ProductName_ID,ProductPackSKU_ID,Point-of-Sale_ID,Date,Measures,Value
0,Family_16,Category_11,ProductBrand_306,ProductName_649,ProductSKU_1970,POS_1,2017-03-04,Sell-out units,2.0
1,Family_16,Category_11,ProductBrand_306,ProductName_649,ProductSKU_1970,POS_1,2017-03-04,Sell-out values,1540.0
2,Family_16,Category_11,ProductBrand_306,ProductName_649,ProductSKU_1970,POS_1,2016-05-02,Sell-out units,4.0
3,Family_16,Category_11,ProductBrand_306,ProductName_649,ProductSKU_1970,POS_1,2016-05-02,Sell-out values,3080.0
4,Family_16,Category_11,ProductBrand_306,ProductName_649,ProductSKU_1970,POS_1,2016-10-24,Sell-out units,2.0
...,...,...,...,...,...,...,...,...,...
182342299,Family_4,Category_34,ProductBrand_279,ProductName_577,ProductSKU_1813,POS_410,2016-04-29,Sell-out units,1.0
182342300,Family_4,Category_34,ProductBrand_279,ProductName_577,ProductSKU_1813,POS_410,2016-04-20,Sell-out values,638.0
182342301,Family_4,Category_34,ProductBrand_279,ProductName_577,ProductSKU_1813,POS_410,2016-04-25,Sell-out values,652.0
182342302,Family_4,Category_34,ProductBrand_279,ProductName_577,ProductSKU_1813,POS_410,2016-04-28,Sell-out values,643.0


In [3]:
#general info
goliath.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 182342304 entries, 0 to 182342303
Data columns (total 9 columns):
 #   Column              Dtype  
---  ------              -----  
 0   ProductFamily_ID    object 
 1   ProductCategory_ID  object 
 2   ProductBrand_ID     object 
 3   ProductName_ID      object 
 4   ProductPackSKU_ID   object 
 5   Point-of-Sale_ID    object 
 6   Date                object 
 7   Measures            object 
 8   Value               float64
dtypes: float64(1), object(8)
memory usage: 12.2+ GB


In [4]:
goliath.isna().sum()

ProductFamily_ID      0
ProductCategory_ID    0
ProductBrand_ID       0
ProductName_ID        0
ProductPackSKU_ID     0
Point-of-Sale_ID      0
Date                  0
Measures              0
Value                 0
dtype: int64

In [5]:
goliath.describe(include = 'all')

,ProductFamily_ID,ProductCategory_ID,ProductBrand_ID,ProductName_ID,ProductPackSKU_ID,Point-of-Sale_ID,Date,Measures,Value
count,182342304,182342304,182342304,182342304,182342304,182342304,182342304,182342304,1.823423e+08
unique,21,178,1523,2820,8509,410,1401,2,NaN
top,Family_12,Category_178,ProductBrand_1425,ProductName_2609,ProductSKU_3008,POS_282,2018-12-10,Sell-out values,NaN
freq,38915420,126256286,2525774,1802618,975138,975220,204254,91171152,NaN
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.760203e+03
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.024838e+03
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1.000000e+01
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000e+00
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.800000e+01
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.654000e+03


### First step: Make it lighter.

We will start by removing all letters from numerically identifiable categories: that is:
ProductFamily_ID	
ProductCategory_ID	
ProductBrand_ID	
ProductName_ID	
ProductPackSKU_ID	

THE SAME PRODUCT MAY HAVE DIFFERENT SKUs!!!!!!

In [6]:
#using regex to all columns to remove unnecessary text

goliath['ProductFamily_ID'] = goliath['ProductFamily_ID'].str.extract('(\d+)', expand=False)
goliath['ProductCategory_ID'] = goliath['ProductCategory_ID'].str.extract('(\d+)', expand=False)
goliath['ProductBrand_ID'] = goliath['ProductBrand_ID'].str.extract('(\d+)', expand=False)
goliath['ProductName_ID'] = goliath['ProductName_ID'].str.extract('(\d+)', expand=False)
goliath['Point-of-Sale_ID'] = goliath['Point-of-Sale_ID'].str.extract('(\d+)', expand=False)
goliath['ProductPackSKU_ID'] = goliath['ProductPackSKU_ID'].str.extract('(\d+)', expand=False)

In [7]:
#convert dataframe to a dataframe half its size by merging values and units on sku, store and data
values_df = goliath[goliath['Measures']=='Sell-out values']
units_df = goliath[goliath['Measures']=='Sell-out units']


goliath = pd.merge(units_df,values_df[['ProductPackSKU_ID','Point-of-Sale_ID','Date','Value']], on=['ProductPackSKU_ID','Point-of-Sale_ID','Date'],suffixes=('_units', '_price'))
goliath.drop(columns='Measures', inplace = True)

In [8]:
goliath.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 91191598 entries, 0 to 91191597
Data columns (total 9 columns):
 #   Column              Dtype  
---  ------              -----  
 0   ProductFamily_ID    object 
 1   ProductCategory_ID  object 
 2   ProductBrand_ID     object 
 3   ProductName_ID      object 
 4   ProductPackSKU_ID   object 
 5   Point-of-Sale_ID    object 
 6   Date                object 
 7   Value_units         float64
 8   Value_price         float64
dtypes: float64(2), object(7)
memory usage: 6.8+ GB


### Additional Feature Engineering

In [9]:
goliath['Unit_Price'] = goliath['Value_price'] / goliath['Value_units']

In [10]:
#This operation will get the max price of each product in each store and use that as the general retail price
goliath['Retail_price'] = goliath.groupby(["ProductPackSKU_ID", "Point-of-Sale_ID"])["Unit_Price"].transform('max')


#Then, create an is_promo column that , if the difference between retail price and unit price (as sold) is larger than 10 %, it was sold on special offer
goliath['Is_Promo'] = np.where(goliath.Unit_Price <= (goliath.Retail_price * 0.9), 1, 0)
goliath

,ProductFamily_ID,ProductCategory_ID,ProductBrand_ID,ProductName_ID,ProductPackSKU_ID,Point-of-Sale_ID,Date,Value_units,Value_price,Unit_Price,Retail_price,Is_Promo
0,16,11,306,649,1970,1,2017-03-04,2.0,1540.0,770.0,810.0,0
1,16,11,306,649,1970,1,2016-05-02,4.0,3080.0,770.0,810.0,0
2,16,11,306,649,1970,1,2016-10-24,2.0,1540.0,770.0,810.0,0
3,16,11,306,649,1970,1,2017-10-13,2.0,1620.0,810.0,810.0,0
4,16,11,306,649,1970,1,2017-10-14,2.0,1620.0,810.0,810.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
91191593,4,34,279,577,1813,410,2016-01-28,1.0,643.0,643.0,810.0,1
91191594,4,34,279,577,1813,410,2016-04-20,1.0,638.0,638.0,810.0,1
91191595,4,34,279,577,1813,410,2016-04-25,1.0,652.0,652.0,810.0,1
91191596,4,34,279,577,1813,410,2016-04-28,1.0,643.0,643.0,810.0,1


In [11]:
#storing as a more manageable CSV to be worked with from now on
goliath.to_csv('../Databases/df_treated.csv')

#### Done

From now on we will always work with df_treated in the future notebooks. 